In [1]:
# # # 1) install deps (run once)
!pip install -q transformers evaluate bert_score datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login

# Method 1: Interactive login (will prompt for token)
login()

In [5]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8bd3d5c0f3396319382977e19cadb9613e52100a1a1e3c7592b351d57c59088c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [6]:
# # # 2) imports
import json
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:




# # # 3) load your patched model exactly as in your file
model_path = "iyashnayi/SocioLens-llama-3.2-3B"   # ← replace
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
model.eval()

# # # 4) embed your prompt_templates dict verbatim
prompt_templates = {
    "analysis": """You are SocioLens — a data-driven and experienced public policy analyst specializing in social impact, governance, and economic equity.

Analyze the following policy issue:
{query}

Approach:
1. **Executive Summary** – Provide a 1-2 sentence summary of the issue and key insight.
2. **Context** – Briefly situate the issue in historical, social, or economic terms.
3. **Key Factors** – Identify driving forces, affected populations, and relevant data indicators.
4. **Stakeholder Impact** – Who gains, who bears the cost, and what power dynamics exist?
5. **Evidence-Based Recommendations** – Provide 2-3 clear policy options with pros/cons.
6. **Implementation Notes** – Mention feasibility, barriers, or trade-offs if applicable.

Keep language clear, neutral, and grounded in data. Highlight uncertainties or assumptions explicitly.""",

    "comparative": """You are SocioLens — a veteran public policy analyst skilled in comparative frameworks and stakeholder-centered evaluation.

Compare the following policy approaches:
{query}

Structure your response as follows:
1. **Policy Snapshot** – Briefly outline each approach in 1–2 sentences.
2. **Evaluation Criteria** – Use 3-4 key axes: effectiveness, cost, equity, and implementation risk.
3. **Stakeholder Analysis** – Discuss who benefits/loses under each option.
4. **Evidence** – Reference real-world data, cases, or trends if possible.
5. **Conclusion** – State which policy fits best in what scenario, and note any hybrid strategies.

Maintain neutrality, present pros/cons clearly, and surface any political or ethical tensions.""",

    "forecast": """You are SocioLens — an experienced policy analyst focused on scenario modeling and systemic forecasting.

Forecast the outcomes of the following policy change:
{query}

Organize your response as follows:
1. **Short-Term Outlook (1–2 yrs)** – Immediate implementation effects and likely reactions.
2. **Medium-Term Outlook (3–5 yrs)** – Shifts in behavior, cost structures, or demographics.
3. **Long-Term Effects (5+ yrs)** – Institutional, environmental, or generational consequences.
4. **Unintended Consequences** – Identify risks, externalities, or vulnerable populations.
5. **Confidence Levels** – Indicate high/medium/low certainty in predictions.
6. **Monitoring Recommendations** – Suggest what metrics or signals should be tracked.

Use a tone that is cautious, analytic, and contingency-aware. Point out where assumptions drive outcomes.""",

    "greeting": """You are SocioLens — a friendly and welcoming public policy assistant.

The user says: "{query}"

Respond with a warm greeting and ask how you can assist, without any policy analysis."""
}

def format_prompt(template_name, query):
    return prompt_templates[template_name].format(query=query)

# # # 5) load your test set
test_path = "/content/policy_training_data (1) (1)_test.jsonl"   # ensure this is in your Colab working dir
test_data = []
with open(test_path) as f:
    for line in f:
        obj = json.loads(line)
        test_data.append({
            "instruction": obj["instruction"],
            "reference":   obj["response"]
        })

# # # 6) prepare evaluation metrics
bleu    = evaluate.load("bleu")
rouge   = evaluate.load("rouge")
bertscr = evaluate.load("bertscore")

results = []

# # # 7) for each template, generate & score
for tpl_name in prompt_templates:
    preds, refs = [], []
    for item in tqdm(test_data, desc=tpl_name):
        prompt = format_prompt(tpl_name, item["instruction"])
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=200)
        txt = tokenizer.decode(out[0], skip_special_tokens=True)
        # strip off prompt
        resp = txt.split(item["instruction"])[-1].strip()
        preds.append(resp)
        refs.append(item["reference"])

    b = bleu.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
    r = rouge.compute(predictions=preds, references=refs)["rougeL"]
    bs = bertscr.compute(predictions=preds, references=refs, lang="en")["f1"]
    results.append({
        "template": tpl_name,
        "BLEU": b,
        "ROUGE-L": r,
        "BERTScore_F1": sum(bs)/len(bs)
    })

# # # 8) save & display
df = pd.DataFrame(results)
df.to_csv("evaluation_metrics_by_template.csv", index=False)
print(df)

# # # 9) grouped bar-chart
metrics = ["BLEU","ROUGE-L","BERTScore_F1"]
x = range(len(prompt_templates))
plt.figure(figsize=(8,5))
for i,m in enumerate(metrics):
    plt.bar([xi + i*0.2 for xi in x],
            df[m],
            width=0.2,
            label=m)
plt.xticks([xi+0.3 for xi in x], df["template"])
plt.ylabel("Score")
plt.title("Performance by Prompt Template")
plt.ylim(0,1)
plt.legend()
plt.tight_layout()
plt.savefig("evaluation_by_template.png")
plt.show()


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from iyashnayi/SocioLens-llama-3.2-3B led to unexpected keys not found in the model: model.layers.0.self_attn.k_proj.lora_A.default.weight, model.layers.0.self_attn.k_proj.lora_B.default.weight, model.layers.0.self_attn.o_proj.lora_A.default.weight, model.layers.0.self_attn.o_proj.lora_B.default.weight, model.layers.1.self_attn.k_proj.lora_A.default.weight, model.layers.1.self_attn.k_proj.lora_B.default.weight, model.layers.1.self_attn.o_proj.lora_A.default.weight, model.layers.1.self_attn.o_proj.lora_B.default.weight, model.layers.2.self_attn.k_proj.lora_A.default.weight, model.layers.2.self_attn.k_proj.lora_B.default.weight, model.layers.2.self_attn.o_proj.lora_A.default.weight, model.layers.2.self_attn.o_proj.lora_B.default.weight, model.layers.3.self_attn.k_proj.lora_A.default.weight, model.layers.3.self_attn.k_proj.lora_B.default.weight, model.layers.3.self_attn.o_proj.lora_A.default.weight, model.layers.3.self_attn.o_proj.lora_B.default.weight, model.layer

analysis:   2%|▏         | 5/243 [00:23<11:39,  2.94s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
analysis:   4%|▎         | 9/243 [00:51<24:40,  6.33s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
analysis:   5%|▍         | 12/243 [01:10<24:47,  6.44s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
analysis:  10%|▉         | 24/243 [02:06<15:59,  4.38s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
analysis:  13%|█▎        | 31/243 [02:35<16:06,  4.56s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for ope